In [1]:
import requests
import time
import pandas as pd

session = requests.Session()
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
    "Referer": "https://store.kyobobook.co.kr/bestseller/person/daily/domestic",
    "Accept": "application/json, text/plain, */*",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7"
}
session.headers.update(headers)

session.get("https://store.kyobobook.co.kr/")  # 쿠키 세팅

author_list = []

for page in range(1, 3): 
    url = f"https://store.kyobobook.co.kr/api/gw/best/best-seller/author?page={page}&per=20&period=001&dsplDvsnCode=001&dsplTrgtDvsnCode=002"
    res = session.get(url)
   

    try:
        data = res.json()
        writers = data.get("data", {}).get("writerList", [])
    except Exception as e:
        print(res.text[:300])
        continue

    if not writers:
        break

    for item in writers:
        info = item.get("writerInfo", {})
        name = info.get("chrcName", "").strip()
        link = f"https://www.kyobobook.co.kr/search/authorSearch.laf?author={name}"

        # 대표 도서 추출
        book_list = item.get("representativeBook", [])
        book_titles = [book.get("cmdtName", "").strip() for book in book_list if book.get("cmdtName")]
        books = ", ".join(book_titles)

        author_list.append({
            "name": name,
            "link": link,
            "books": books
        })

    time.sleep(1)

# 저장 및 출력
df = pd.DataFrame(author_list)


df.head()


,name,link,books
0,홍범준,https://www.kyobobook.co.kr/search/authorSearc...,"쎈B 초등 수학 3-2(2025), 일품 중등 수학 1-1 462제(2025), 쎈..."
1,이재명,https://www.kyobobook.co.kr/search/authorSearc...,"결국 국민이 합니다, 함께 가는 길은 외롭지 않습니다, 그 꿈이 있어 여기까지 왔다"
2,한강,https://www.kyobobook.co.kr/search/authorSearc...,"흰, Lecons de Grec, Ces soirs ranges dans mon t..."
3,김영하,https://www.kyobobook.co.kr/search/authorSearc...,"단 한 번의 삶, 김영하의 세계문학 원정대 4: 빨간 머리 앤, 김영하의 세계문학 ..."
4,David Cho,https://www.kyobobook.co.kr/search/authorSearc...,해커스 토플 스피킹 인터미디엇(Hackers TOEFL Speaking Interm...
